<a href="https://colab.research.google.com/github/tejaswi0910/CommunityClassroom-Git/blob/master/Kyber%2BProposal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth=1 https://github.com/open-quantum-safe/liboqs-python

Cloning into 'liboqs-python'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 33 (delta 3), reused 17 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 25.64 KiB | 1.35 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!apt-get update
!apt-get install -y cmake ninja-build gcc make python3-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [66.7 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/

In [ ]:
%cd liboqs-python
!pip install .

/content/liboqs-python
Processing /content/liboqs-python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for liboqs-python: filename=liboqs_python-0.12.0-py3-none-any.whl size=12437 sha256=2417e7d315c2b05e79d367af0717f97ea0b3aeb82f16df9837a0bb427a6e3092
  Stored in directory: /root/.cache/pip/wheels/88/6a/34/7b907e7c67601095491eb47ea991c49facb1ce071d5b40d09c
Successfully built liboqs-python


In [ ]:
import oqs

liboqs not found, installing it in /root/_oqs


INFO:oqs.oqs:liboqs not found, installing it in /root/_oqs


Installing in 5 seconds...


INFO:oqs.oqs:Installing in 5 seconds...


Installing in 4 seconds...


...

INFO:oqs.oqs:Installing in 3 seconds...


Installing in 2 seconds...


INFO:oqs.oqs:Installing in 2 seconds...


Installing in 1 seconds...


INFO:oqs.oqs:Installing in 1 seconds...


In [ ]:
!pip install pycryptodome

## Kyber+

Key Enhancements:
- Multithreading: Threads are used to parallelize the key generation, encryption, and decryption processes for different algorithms, improving performance.
- Hybrid Encryption: Combines Kyber for key exchange with AES for encrypting the plaintext message.
- Batch Processing: Although this example processes algorithms sequentially within threads, you could extend it to handle multiple messages in batches.
- Input/Output text


ToDo:
- graphs - Done
- figure out a metric for loss - Shanon Entropy, Chi Square, Hamming
- figure why relation isn't consistent

## Kyber’s Role (Key Encapsulation Mechanism - KEM)
Kyber is a post-quantum key encapsulation mechanism (KEM), which means it does not encrypt the plaintext directly.

Instead, it is used for secure key exchange, meaning:
- A public and private key pair is generated using kem.generate_keypair(). The shared secret is encapsulated using kem.encap_secret(public_key), producing:
  - A ciphertext (used for key exchange)
  - A shared secret (used as an encryption key)
  - The receiver decapsulates the shared secret using kem.decap_secret(ciphertext), retrieving the same shared secret.
- This ensures quantum-safe key exchange between two parties.

---

## AES’s Role (Hybrid Encryption of the Actual Message)
After obtaining the shared secret from Kyber, it is used as the encryption key for AES. AES is a symmetric encryption algorithm, meaning the same key is used for both encryption and decryption.

The steps for AES encryption:
 - Take the first 16 bytes of Kyber’s shared secret as the AES key.
 - Generate a random initialization vector (IV).
 - Use AES in CBC mode to encrypt the actual plaintext message.
 - The IV is prepended to the ciphertext, so it can be used for decryption.

The steps for AES decryption:
- Extract the first 16 bytes of the ciphertext as the IV.
- Decrypt the message using the same shared secret as the AES key.

In [ ]:
# Function to read a text file and extract sentences
def read_text_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()  # Read the entire content

    # Split text into sentences using a full stop (.)
    sentences = text.split(".")

    # Remove empty lines, extra spaces, and line breaks
    sentences = [sentence.strip().replace("\n", " ") for sentence in sentences if sentence.strip()]

    return sentences

# Path to your text file
file_path1 = "/content/plaintext-small.txt"
file_path2 = "/content/plaintext-large.txt"

# Read and process the file
sentence_list_small = read_text_file(file_path1)[:120]
sentence_list_large = read_text_file(file_path2)[:600]

# Print the cleaned list of sentences
# print(sentence_list)
print(len(sentence_list_small))
print(len(sentence_list_large))

In [ ]:
import time
import oqs
import threading
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os
from collections import Counter
import math

def shannon_entropy(data):
    if not data:
        return 0
    frequency = Counter(data)  # Count occurrences of each byte
    total_length = len(data)
    entropy = -sum((count / total_length) * math.log2(count / total_length) for count in frequency.values())
    return entropy

# Hybrid encryption: AES with Kyber for key exchange
def hybrid_encrypt(shared_secret, plaintext):
    aes_key = shared_secret[:16]  # Use the first 16 bytes for AES key
    iv = os.urandom(16)  # Random initialization vector
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    return iv + ciphertext

def hybrid_decrypt(shared_secret, ciphertext):
    aes_key = shared_secret[:16]
    iv = ciphertext[:16]
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[16:]), AES.block_size)
    return plaintext

# Multithreaded key generation, encryption, and decryption
def process_kyber(algo, plaintext):
    results = {}
    keygen_time = 0
    enc_time = 0
    dec_time = 0
    ciphertext_entropy = 0
    encrypted_entropy = 0

    with oqs.KeyEncapsulation(algo) as kem:
        for sentence in plaintext:
            pt = sentence.encode('utf-8')
            # Key generation
            start_time = time.time()
            public_key = kem.generate_keypair()
            # private_key = kem.export_secret_key()
            keygen_time += time.time() - start_time

            # Encryption
            start_time = time.time()
            ciphertext, shared_secret = kem.encap_secret(public_key)
            enc_time += time.time() - start_time
            ciphertext_entropy += shannon_entropy(ciphertext)

            # Decryption
            start_time = time.time()
            shared_secret_dec = kem.decap_secret(ciphertext)
            dec_time += time.time() - start_time

            # Hybrid encryption and decryption
            encrypted_message = hybrid_encrypt(shared_secret, pt)
            decrypted_message = hybrid_decrypt(shared_secret_dec, encrypted_message)
            encrypted_entropy += shannon_entropy(encrypted_message)

        n = len(plaintext)
        results = {
            "keygen_time": keygen_time / n,
            "enc_time": enc_time / n,
            "dec_time": dec_time / n,
            "encrypted_message": encrypted_message,
            "decrypted_message": decrypted_message,
            "ciphertext_entropy": ciphertext_entropy / n,
            "entropy": encrypted_entropy / n
        }

    return results

# Example plaintext to encrypt
plaintext = sentence_list_small

# Define the algorithms
algorithms = ["Kyber512", "Kyber768", "Kyber1024"]

# Multithreaded execution
threads = []
results = []

def thread_function(algo, plaintext):
    result = process_kyber(algo, plaintext)
    result["algorithm"] = algo
    results.append(result)

for algo in algorithms:
    thread = threading.Thread(target=thread_function, args=(algo, plaintext))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

# Display the results
for result in results:
    print(f"Algorithm: {result['algorithm']}")
    print(f"Key Generation Time: {result['keygen_time']:.6f} seconds")
    print(f"Encryption Time: {result['enc_time']:.6f} seconds")
    print(f"Decryption Time: {result['dec_time']:.6f} seconds")
    print(f"Encrypted Message: {result['encrypted_message']}")
    print(f"Decrypted Message: {result['decrypted_message']}")
    print(f"Shanon Entropy on CipherText: {result['ciphertext_entropy']}")
    print(f"Shanon Entropy on Encrypted Message: {result['entropy']}")
    print("-" * 30)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'results' is already populated after running the algorithms
# Prepare data for plotting
metrics = ["keygen_time", "enc_time", "dec_time", "ciphertext_entropy", "entropy"]
algorithms = ["Kyber512", "Kyber768", "Kyber1024"]

# Extract the relevant results for each algorithm
keygen_times = [result["keygen_time"] for result in results]
enc_times = [result["enc_time"] for result in results]
dec_times = [result["dec_time"] for result in results]
ciphertext_entropies = [result["ciphertext_entropy"] for result in results]
encrypted_entropies = [result["entropy"] for result in results]

# Create a 3x2 grid of subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 10))

# Plot each metric
axes[0, 0].bar(algorithms, keygen_times, color=['#6C5B7B', '#F1A7B9', '#F6D02F'])
axes[0, 0].set_title('Key Generation Time')
axes[0, 0].set_ylabel('Time (seconds)')

axes[0, 1].bar(algorithms, enc_times, color=['#6C5B7B', '#F1A7B9', '#F6D02F'])
axes[0, 1].set_title('Encryption Time')
axes[0, 1].set_ylabel('Time (seconds)')

axes[1, 0].bar(algorithms, dec_times, color=['#6C5B7B', '#F1A7B9', '#F6D02F'])
axes[1, 0].set_title('Decryption Time')
axes[1, 0].set_ylabel('Time (seconds)')

axes[1, 1].bar(algorithms, ciphertext_entropies, color=['#6C5B7B', '#F1A7B9', '#F6D02F'])
axes[1, 1].set_title('Ciphertext Entropy')
axes[1, 1].set_ylabel('Entropy')

axes[2, 0].bar(algorithms, encrypted_entropies, color=['#6C5B7B', '#F1A7B9', '#F6D02F'])
axes[2, 0].set_title('Encrypted Message Entropy')
axes[2, 0].set_ylabel('Entropy')

# Hide the empty subplot
axes[2, 1].axis('off')

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plot
plt.show()

600 sentences + chi quare +

In [ ]:
import time
import oqs
import threading
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os
from collections import Counter
import math

def shannon_entropy(data):
    if not data:
        return 0
    frequency = Counter(data)  # Count occurrences of each byte
    total_length = len(data)
    entropy = -sum((count / total_length) * math.log2(count / total_length) for count in frequency.values())
    return entropy

# Chi-Square Test Calculation
def chi_square_test(data):
    expected_freq = len(data) / 256  # Expected uniform distribution
    observed_freq = Counter(data)  # Count occurrences of each byte

    chi_square_stat = sum(((observed_freq.get(byte, 0) - expected_freq) ** 2) / expected_freq for byte in range(256))
    return chi_square_stat

# Hamming Distance Calculation
def hamming_distance(data1, data2):
    """Compute Hamming Distance between two byte sequences."""
    return sum(bin(b1 ^ b2).count('1') for b1, b2 in zip(data1, data2))


# Hybrid encryption: AES with Kyber for key exchange
def hybrid_encrypt(shared_secret, plaintext):
    aes_key = shared_secret[:16]  # Use the first 16 bytes for AES key
    iv = os.urandom(16)  # Random initialization vector
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    return iv + ciphertext

def hybrid_decrypt(shared_secret, ciphertext):
    aes_key = shared_secret[:16]
    iv = ciphertext[:16]
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[16:]), AES.block_size)
    return plaintext

# Multithreaded key generation, encryption, and decryption
def process_kyber(algo, plaintext):
    results = {}
    keygen_time = 0
    enc_time = 0
    dec_time = 0
    ciphertext_entropy = 0
    encrypted_entropy = 0
    ciphertext_chi_square = 0
    encrypted_chi_square = 0

    with oqs.KeyEncapsulation(algo) as kem:
        for sentence in plaintext:
            pt = sentence.encode('utf-8')
            # Key generation
            start_time = time.time()
            public_key = kem.generate_keypair()
            # private_key = kem.export_secret_key()
            keygen_time += time.time() - start_time

            # Encryption
            start_time = time.time()
            ciphertext, shared_secret = kem.encap_secret(public_key)
            enc_time += time.time() - start_time
            ciphertext_entropy += shannon_entropy(ciphertext)
            ciphertext_chi_square += chi_square_test(ciphertext)

            # Decryption
            start_time = time.time()
            shared_secret_dec = kem.decap_secret(ciphertext)
            dec_time += time.time() - start_time

            # Hybrid encryption and decryption
            encrypted_message = hybrid_encrypt(shared_secret, pt)
            decrypted_message = hybrid_decrypt(shared_secret_dec, encrypted_message)
            encrypted_entropy += shannon_entropy(encrypted_message)
            encrypted_chi_square += chi_square_test(ciphertext)


        n = len(plaintext)
        results = {
            "keygen_time": keygen_time / n,
            "enc_time": enc_time / n,
            "dec_time": dec_time / n,
            "encrypted_message": encrypted_message,
            "decrypted_message": decrypted_message,
            "ciphertext_entropy": ciphertext_entropy / n,
            "entropy": encrypted_entropy / n,
            "ciphertext_chi_square": ciphertext_chi_square / n,
            "encrypted_chi_square": encrypted_chi_square / n,
            # "hamming_distance": hamming_distance(ciphertext, encrypted_message) / n"
        }

    return results

# Example plaintext to encrypt
plaintext = sentence_list_large

# Define the algorithms
algorithms = ["Kyber512", "Kyber768", "Kyber1024"]

# Multithreaded execution
threads = []
results = []

def thread_function(algo, plaintext):
    result = process_kyber(algo, plaintext)
    result["algorithm"] = algo
    results.append(result)

for algo in algorithms:
    thread = threading.Thread(target=thread_function, args=(algo, plaintext))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

# Display the results
for result in results:
    print(f"Algorithm: {result['algorithm']}")
    print(f"Key Generation Time: {result['keygen_time']:.6f} seconds")
    print(f"Encryption Time: {result['enc_time']:.6f} seconds")
    print(f"Decryption Time: {result['dec_time']:.6f} seconds")
    print(f"Encrypted Message: {result['encrypted_message']}")
    print(f"Decrypted Message: {result['decrypted_message']}")
    print(f"Shanon Entropy on CipherText: {result['ciphertext_entropy']}")
    print(f"Shanon Entropy on Encrypted Message: {result['entropy']}")
    print(f"Chi Square on ciphertext: {result['ciphertext_chi_square']}")
    print(f"Chi Suare on encrypted message: {result['encrypted_chi_square']}")

    print("-" * 30)

Algorithm: Kyber768
Key Generation Time: 0.000425 seconds
Encryption Time: 0.000305 seconds
Decryption Time: 0.000465 seconds
Encrypted Message: b'M\x81\xb3%,a\x9b\xb5\x0f\xfc\xe1\x89"\x8dd^\x93\x9a8\xedW\xb3\xa2\x1e\x8b\x83[\x02P\xc1\x07\x82\xa4A+\x8f\xb1\xe8\xffA\xd5\x8d\xf1\xd1\xaaw\x90-w\x94a\xb64\x82 \x0f\x0b\x1f\xb2\xf2\xdf\xc5\xcd\xa2\x86\xee\xfaQ\xf3\x97\xad\x92y\x04\'\xa7\x98\xe1\x13\xf7\x9f(\xe0\xf9^\x08\x9d\xfb\xdb\xabS\xd8cX\xed0\xc8\x86\xdd\xe8jsG\x9f\xc2\x01JqA\xae\xe7\x9f'
Decrypted Message: b'You just stood there with your red plastic tray like there was this awful spotlight on you'
Shanon Entropy on CipherText: 7.820433679449091
Shanon Entropy on Encrypted Message: 6.260372851677395
Chi Square on ciphertext: 255.6650980392155
Chi Suare on encrypted message: 255.6650980392155
------------------------------
Algorithm: Kyber1024
Key Generation Time: 0.000436 seconds
Encryption Time: 0.000302 seconds
Decryption Time: 0.000419 seconds
Encrypted Message: b'\xe9j\xc0W\x1a\xfe